In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

pd.set_option('display.max_colwidth', 200)
# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

In [2]:
# !pip install -q efficientnet

In [3]:
train = pd.read_csv('../input/cub200-2011-csv/train.csv')
test = pd.read_csv('../input/cub200-2011-csv/test.csv')

print(train.shape, test.shape)
train.head()

(5994, 10) (5794, 10)


,ImageID,x,y,width,height,filename,Category,is_training,Class Name,Label
0,2,139.0,30.0,153.0,264.0,001.Black_footed_Albatross/Black_Footed_Albatross_0009_34.jpg,1,1,001.Black_footed_Albatross,0
1,4,112.0,90.0,255.0,242.0,001.Black_footed_Albatross/Black_Footed_Albatross_0074_59.jpg,1,1,001.Black_footed_Albatross,0
2,5,70.0,50.0,134.0,303.0,001.Black_footed_Albatross/Black_Footed_Albatross_0014_89.jpg,1,1,001.Black_footed_Albatross,0
3,7,7.0,75.0,420.0,262.0,001.Black_footed_Albatross/Black_Footed_Albatross_0031_100.jpg,1,1,001.Black_footed_Albatross,0
4,8,78.0,86.0,333.0,158.0,001.Black_footed_Albatross/Black_Footed_Albatross_0051_796103.jpg,1,1,001.Black_footed_Albatross,0


In [4]:
tpath = '../input/cub200/CUB_200_2011/images'
train_paths = train.filename.apply(lambda x: os.path.join(tpath, x))
train_paths.values

array(['../input/cub200/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0009_34.jpg',
       '../input/cub200/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0074_59.jpg',
       '../input/cub200/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0014_89.jpg',
       ...,
       '../input/cub200/CUB_200_2011/images/200.Common_Yellowthroat/Common_Yellowthroat_0094_190690.jpg',
       '../input/cub200/CUB_200_2011/images/200.Common_Yellowthroat/Common_Yellowthroat_0037_190698.jpg',
       '../input/cub200/CUB_200_2011/images/200.Common_Yellowthroat/Common_Yellowthroat_0049_190708.jpg'],
      dtype=object)

In [5]:
from tensorflow.keras.utils import to_categorical

# convert to one-hot-encoding-labels
train_labels = to_categorical(train.Label)
train_labels.shape

(5994, 200)

In [6]:
tpath = '../input/cub200/CUB_200_2011/images'
test_paths = test.filename.apply(lambda x: os.path.join(tpath, x))
test_paths.values

array(['../input/cub200/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg',
       '../input/cub200/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0002_55.jpg',
       '../input/cub200/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0085_92.jpg',
       ...,
       '../input/cub200/CUB_200_2011/images/200.Common_Yellowthroat/Common_Yellowthroat_0058_190958.jpg',
       '../input/cub200/CUB_200_2011/images/200.Common_Yellowthroat/Common_Yellowthroat_0008_190703.jpg',
       '../input/cub200/CUB_200_2011/images/200.Common_Yellowthroat/Common_Yellowthroat_0055_190967.jpg'],
      dtype=object)

In [7]:
def decode_image(filename, label=None, image_size=(299, 299)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    
    if label is None:
        return image
    else:
        return image, label

In [8]:
def augment(image, label=None):
    image = tf.image.random_flip_left_right(image)
    
    if label is None:
        return image
    else:
        return image, label

In [9]:
batch_size = 32

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_paths, train_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .map(augment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(2048)
    .batch(batch_size)
    .prefetch(AUTO)
)

test_dataset = (tf.data.Dataset
        .from_tensor_slices(test_paths)
        .map(decode_image, num_parallel_calls=AUTO)
        .batch(batch_size)
)

In [10]:
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# from efficientnet.tfkeras import EfficientNetB2

model = tf.keras.Sequential([
        ResNet101(weights = 'imagenet', 
                       include_top = False,
                       input_shape = (299, 299, 3)),
        GlobalAveragePooling2D(),
        Dense(1024, activation = 'relu'),
        Dropout(0.25),
        Dense(512, activation = 'relu'),
        Dropout(0.25),
        Dense(200, activation = 'softmax')
])
    
model.compile(optimizer = 'sgd',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
    
model.summary()

171450368/171446536 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet101 (Model)            (None, 10, 10, 2048)      42658176  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_______________________________________________________

In [11]:
def Train_model(model, batch_size, EPOCHS):
    n_steps = train_labels.shape[0] // batch_size
    EPOCHS = EPOCHS
    
    Model = model
    history = Model.fit(train_dataset, 
                    steps_per_epoch = n_steps,
                    epochs = EPOCHS,
                    verbose = 1)
    return Model

In [12]:
batch_size = 32
EPOCHS = 50

def check(x, y):
    if x == y:
        return 1
    else:
        return 0
    
print('Training')
model = Train_model(model, batch_size, EPOCHS)
pred = model.predict(test_dataset, verbose=1)
    
test['Prediction'] = pred.argmax(axis=1)
test['Score'] = test.apply(lambda x: check(x['Prediction'], x['Label']), axis=1)
print('Test accuracy on iterations is ', 100 * test.Score.sum() / test.shape[0])

Training
Epoch 1/50
187/187 [==============================] - 94s 505ms/step - loss: 5.1743 - accuracy: 0.0296
Epoch 2/50
187/187 [==============================] - 94s 504ms/step - loss: 4.4297 - accuracy: 0.1156
Epoch 3/50
187/187 [==============================] - 94s 503ms/step - loss: 3.3167 - accuracy: 0.2652
Epoch 4/50
187/187 [==============================] - 94s 504ms/step - loss: 2.3931 - accuracy: 0.4083
Epoch 5/50
187/187 [==============================] - 95s 506ms/step - loss: 1.6980 - accuracy: 0.5521
Epoch 6/50
187/187 [==============================] - 94s 504ms/step - loss: 1.2604 - accuracy: 0.6591
Epoch 7/50
187/187 [==============================] - 94s 503ms/step - loss: 0.9452 - accuracy: 0.7311
Epoch 8/50
187/187 [==============================] - 94s 504ms/step - loss: 0.6948 - accuracy: 0.8045
Epoch 9/50
187/187 [==============================] - 94s 504ms/step - loss: 0.5281 - accuracy: 0.8493
Epoch 10/50
187/187 [==============================] - 94s 503ms

In [14]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(test.Label, test.Prediction))

              precision    recall  f1-score   support

           0       0.79      0.77      0.78        30
           1       0.81      0.70      0.75        30
           2       0.70      0.93      0.80        28
           3       0.78      0.83      0.81        30
           4       1.00      0.86      0.92        14
           5       0.83      0.91      0.87        11
           6       0.88      0.96      0.92        23
           7       0.70      0.78      0.74        18
           8       0.67      0.55      0.60        29
           9       0.84      0.90      0.87        30
          10       0.57      0.43      0.49        30
          11       0.91      0.81      0.86        26
          12       0.88      0.73      0.80        30
          13       0.85      0.97      0.91        30
          14       0.92      0.82      0.87        28
          15       0.96      0.89      0.93        28
          16       0.87      0.96      0.91        27
          17       0.94    